In [7]:
from glob import glob
from pprint import pprint
video_paths = sorted(glob("/home/agent_h/data/datasets/facetalk_lrs3_v2/video/test_000000/*.mp4"))
text_paths = sorted(glob("/home/agent_h/data/datasets/facetalk_lrs3_v2/text/test_000000/*.txt"))

print(len(video_paths))
print(video_paths[:10])

          

1321
['/home/agent_h/data/datasets/facetalk_lrs3_v2/video/test_000000/0Fi83BHQsMA_00002.mp4', '/home/agent_h/data/datasets/facetalk_lrs3_v2/video/test_000000/0Fi83BHQsMA_00004.mp4', '/home/agent_h/data/datasets/facetalk_lrs3_v2/video/test_000000/0Fi83BHQsMA_00005.mp4', '/home/agent_h/data/datasets/facetalk_lrs3_v2/video/test_000000/0Fi83BHQsMA_00006.mp4', '/home/agent_h/data/datasets/facetalk_lrs3_v2/video/test_000000/0QVXdEOiCw8_00001.mp4', '/home/agent_h/data/datasets/facetalk_lrs3_v2/video/test_000000/0VJqrlH9cdI_00001.mp4', '/home/agent_h/data/datasets/facetalk_lrs3_v2/video/test_000000/0VJqrlH9cdI_00002.mp4', '/home/agent_h/data/datasets/facetalk_lrs3_v2/video/test_000000/0ZfSOArXbGQ_00001.mp4', '/home/agent_h/data/datasets/facetalk_lrs3_v2/video/test_000000/0ZfSOArXbGQ_00002.mp4', '/home/agent_h/data/datasets/facetalk_lrs3_v2/video/test_000000/0ZfSOArXbGQ_00003.mp4']


In [14]:
import pandas
csv_path = "/home/agent_h/data/datasets/autoavsr_lrs3_train/labels/lrs3_train_transcript_lengths_seg16s.csv"
csv_path = "/home/agent_h/data/datasets/autoavsr_lrs3_test/labels/lrs3_test_transcript_lengths_seg16s.csv"

tmp_df = pandas.read_csv(csv_path)
dicts = tmp_df.to_dict('records')

In [17]:
print(dicts[0])
lengths = [x['27'] for x in dicts]
print(min(lengths))

{'lrs3': 'lrs3', 'lrs3_video_seg16s/test/Sbm98rrVZzE/00001.mp4': 'lrs3_video_seg16s/test/YyXRYgjQXX0/00013.mp4', '27': 67, '1063 2550 3231 5037 2456': '4590 2 302 4056 2550 2 309 4469 1066 544 1054 3327 4244 407 713 4154 407'}
14


In [2]:
video_path = "/home/agent_h/data/datasets/facetalk_lrs3_v2/video/test_000000/0ZfSOArXbGQ_00003.mp4"
for i in range(64):
    # print(f"python preprocess_lrs2lrs3.py --data-dir /home/agent_h/data/datasets/lrs3/ --landmarks-dir /home/agent_h/data/datasets/LRS3_landmarks/ --detector retinaface --root-dir /home/agent_h/data/datasets/autoavsr_lrs3 --dataset lrs3 --gpu_type cuda --subset train --seg-duration 16 --groups 64 --job-index {i} &")
    print(f"CUDA_VISIBLE_DEVICES={i%8} python preprocess_lrs2lrs3_v2.py --data-dir /home/agent_h/data/datasets/lrs3_bili_splitted/ --detector retinaface --root-dir /home/agent_h/data/datasets/autoavsr_bili_splitted/ --dataset lrs3 --gpu_type cuda --subset test --seg-duration 16 --groups 64 --job-index {i} &")

    

CUDA_VISIBLE_DEVICES=0 python preprocess_lrs2lrs3_v2.py --data-dir /home/agent_h/data/datasets/lrs3_bili_splitted/ --detector retinaface --root-dir /home/agent_h/data/datasets/autoavsr_bili_splitted/ --dataset lrs3 --gpu_type cuda --subset test --seg-duration 16 --groups 64 --job-index 0 &
CUDA_VISIBLE_DEVICES=1 python preprocess_lrs2lrs3_v2.py --data-dir /home/agent_h/data/datasets/lrs3_bili_splitted/ --detector retinaface --root-dir /home/agent_h/data/datasets/autoavsr_bili_splitted/ --dataset lrs3 --gpu_type cuda --subset test --seg-duration 16 --groups 64 --job-index 1 &
CUDA_VISIBLE_DEVICES=2 python preprocess_lrs2lrs3_v2.py --data-dir /home/agent_h/data/datasets/lrs3_bili_splitted/ --detector retinaface --root-dir /home/agent_h/data/datasets/autoavsr_bili_splitted/ --dataset lrs3 --gpu_type cuda --subset test --seg-duration 16 --groups 64 --job-index 2 &
CUDA_VISIBLE_DEVICES=3 python preprocess_lrs2lrs3_v2.py --data-dir /home/agent_h/data/datasets/lrs3_bili_splitted/ --detector r

In [2]:
import decord
from decord import VideoReader, gpu
import numpy as np

# Initialize the video reader with GPU context
def fast_video_read_gpu(video_path, device_id=0):
    vr = VideoReader(video_path, ctx=gpu(device_id))

    # Retrieve all frames in a single batch
    # (You can also slice or pick specific indices for partial reading)
    frames = vr.get_batch(range(len(vr)))

    # frames is a decord.VideoLoaderNDArray: shape [T, H, W, C]
    # by default, decord loads in HWC with RGB pixel format

    # If you need a NumPy array on the CPU:
    frames_np = frames.asnumpy()  # shape: (T, H, W, 3)

    return frames_np
fast_video_read_gpu("/home/agent_h/data/datasets/facetalk_lrs3_v2/video/test_000000/0Fi83BHQsMA_00002.mp4")

DECORDError: [23:18:10] /github/workspace/src/video/video_reader.cc:167: CUDA not enabled. Requested context GPU(0).

In [5]:
pip install decord-cu1

ERROR: Could not find a version that satisfies the requirement decord-cu123 (from versions: none)
ERROR: No matching distribution found for decord-cu123
Note: you may need to restart the kernel to use updated packages.


In [10]:
# clean data
from glob import glob
from tqdm import tqdm
all_txt_paths = glob("/home/agent_h/data/datasets/lrs3_bili_splitted/**/*.txt",recursive=True)
print(all_txt_paths[0])
for txt_path in tqdm(all_txt_paths):
    with open(txt_path) as f:
        lines = f.readlines()
    # print(lines)
    new_lines = ""
    for line_idx,line in enumerate(lines):
        line = "".join(ch for ch in line if ch.isascii())
        if line_idx <= 2:
            new_lines += line.upper()
        else:
            data = line.split(" ")
            if len(data) == 4:
                new_lines += line.upper()
            if len(data) !=4:
                # # print(data)
                word = "".join(data[:-3])
                word = word.replace(" ","")
                if word:
                    new_line = " ".join([word,data[-3],data[-2],data[-1]])
                    new_lines += new_line.upper()
             
        # break
            
    # print(new_lines)
    with open(txt_path, 'w', encoding='utf-8') as f:
        f.write(new_lines)
    # break
    
                            




/home/agent_h/data/datasets/lrs3_bili_splitted/trainval/202307111016_bilibili_afeikathy_002_000011/202307111016_bilibili_afeikathy_002_000011.txt


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 238526/238526 [00:34<00:00, 6958.53it/s]


In [3]:
# sampleing
from tqdm import tqdm
import random
from glob import glob
import os
target = "/home/agent_h/data/datasets/lrs3_bili_splitted_sampled/test"
for path in tqdm(glob("/home/agent_h/data/datasets/lrs3_bili_splitted/test/*")):
    if random.random()<0.1:
        os.system("cp -r {} {}".format(path,path.replace("/lrs3_bili_splitted/","/lrs3_bili_splitted_sampled/")))


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4842/4842 [00:04<00:00, 994.46it/s]


In [34]:
# generate cropus
from glob import glob
from tqdm import tqdm

all_text_paths = glob("/home/agent_h/data/datasets/lrs3_bili_splitted/**/*.txt",recursive=True)
print(len(all_text_paths))
corpus_lines = ""
for path in tqdm(all_text_paths):
    with open(path) as f:
        lines = f.readlines()
    corpus_lines+=lines[0][6:]

with open("../spm2/input.txt", 'w', encoding='utf-8') as f:
    f.write(corpus_lines)

47498


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 47498/47498 [00:23<00:00, 2026.26it/s]


[' jiu hui kandao yixie bi jiao ke yu zhi de bu xuyao zheme duo gai bian de yixie huan jing le bu xuyao rang ziji jing chang bei zhe xie gai bian qu xiao hao nide neng liang zui hou women lai kan ruguo shi ying li chu xianzai le nide lie shi\n', ' ranhou wang xia suan zhege zhong jie zhege fu kuan fang shi ne yeshi keyi he zhong jie tan de dui ranhou wojuede shi xi sheng shi zui hao ya yi fu yi ba bu ran ni zhong tu pao\n', ' wu fa ti dai ren lei xie zuo de muqian de AI ying gai neng zuo dao shou ji zheng li jiu cuo de yixie neng li huozhe shuo xiang zhi neng ji qi ren yi yang jin xing\n', ' yao guo du de da tan dui fang de jia ting lai cha shou dui fang de jia wu shi nishi ni tashi ta bao chi bi yao de jie xian gan mingbai le ma ruguo dou neng zuo dao name\n', ' yinwei ta guo de shenghuo qishi shiwo da xue shi qi de meng xiao shihou kan lu xun ta jiu ling shuo le gao zhong deshihou wojiu kan le bu shao xiang shenme kong qing dong  yu jie  qian li xun zhi lei de shu zhege da xue shi re